# Tutorial 1 <br /> <br /> Solve Dantzig's Transport Problem using the *ix modeling platform* (ixmp4)

<img style="float: right; height: 80px;" src="_static/python.png">

## Aim and scope of the tutorial

This tutorial takes you through the steps to import the data for a very simple optimization model
and solve it using the **ixmp4**-GAMS interface.

We use Dantzig's transport problem, which is also used as the standard GAMS tutorial.
This problem finds a least cost shipping schedule that meets requirements at markets and supplies at factories.

If you are not familiar with GAMS, please take a minute to look at the [transport.gms](transport.gms) code.

For reference of the transport problem, see:
> Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.  
> Princeton University Press, Princeton, New Jersey, 1963.

> This formulation is described in detail in:  
> Rosenthal, R E, Chapter 2: A GAMS Tutorial.  
> In GAMS: A User's Guide. The Scientific Press, Redwood City, California, 1988.

> see http://www.gams.com/mccarl/trnsport.gms

## Tutorial outline

The steps in the tutorial are the following:

0. Launch an `ixmp4.Platform` instance and initialize a new `ixmp4.Run`.
0. Define the `Set`s and `Parameter`s in the scenario and save the data to the platform.
0. Initialize `Variable`s and `Equation`s to import the solution from GAMS.
0. Call GAMS to **solve the scenario** (export to GAMS input gdx, execute, read solution from output gdx).
0. Display the **solution** (variables and equations).

## Launching the `Platform` and initializing a new `Run`

A `Platform` is the connection to the database that holds all data and relevant additional information.

A `Run` is an object that holds all relevant information for one quantification of a scenario.  
A run is identified by a model name, a scenario name and a version number (assigned automatically).

You have to *register a new local database* before you can run the tutorial. 

Run the following in the command-line:
```
ixmp4 platforms add tutorial-test
```

You can then check if the database was successfully created by running
```
ixmp4 platforms list
```

After creating the database, you can connect to it via an `ixmp4.Platform` instance.

In [ ]:
import ixmp4

platform = ixmp4.Platform("tutorial-test")

Now, we initialize a new `ixmp4.Run` in the database.

In [ ]:
run = platform.runs.create(model="transport problem", scenario="standard")

## Defining the `IndexSet`s

An `IndexSet` defines a list of elements with a name. These `IndexSet`s can be used for "indexed assignment" of parameters, variables and equations. 
The entries of these parameters, etc. are then validated against the elements of the linked `IndexSet`(s). 
In database terms, a column of a parameter can be "foreign-keyed" onto an `IndexSet`.

Below, we first show the data as they would be written in the GAMS tutorial ([transport.gms](transport.gms) in this folder).  

We now initialize these sets and assign the elements.

In [ ]:
i = run.optimization.indexsets.create("i")
i.add(["seattle", "san-diego"])

We can display the elements of any `IndexSet` as a Python list:

In [ ]:
i.elements

`IndexSet`s can be notated with documentation strings to record their meaning. These strings can be translated to GAMS, too! 

In [ ]:
i.docs = "Canning Plants"

For simplicity, the steps of creating an `IndexSet` and assigning elements can also be done in one line:

In [ ]:
run.optimization.indexsets.create("j").add(["new-york", "chicago", "topeka"])

## Assigning the `Parameter`s

Next, we define the parameters *capacity* and *demand*. The parameters are assigned on the indexsets *i* and *j*, respectively.

In [ ]:
import pandas as pd

from ixmp4.core import Unit

# We only need to (and can!) define units once for each Platform
try:
    cases = platform.units.get("cases")
except Unit.NotFound:
    cases = platform.units.create("cases")

# Capacity of plant i in cases
a = run.optimization.parameters.create(name="a", constrained_to_indexsets=["i"])
# Parameter data can be a dict...
a_data = {
    "i": ["seattle", "san-diego"],
    "values": [350, 600],
    "units": [cases.name, cases.name],
}
a.add(data=a_data)

# Demand at market j in cases
b = run.optimization.parameters.create("b", constrained_to_indexsets=["j"])
# ... or a pd.DataFrame
b_data = pd.DataFrame(
    [
        ["new-york", 325, cases.name],
        ["chicago", 300, cases.name],
        ["topeka", 275, cases.name],
    ],
    columns=["j", "values", "units"],
)
b.add(b_data)

Notice how the `parameter.data` has three columns but has only been linked to one `IndexSet`? That's on purpose: Every `Parameter` needs to have (the columns) *values* and *units*, but these cannot be constrained to an `IndexSet`. The value(s) can be any number(s), but the units need to be known to the `Platform`.

Here's how to access `parameter.data` to e.g. quickly confirm that *b* is set correctly:

In [ ]:
b.data

In [ ]:
try:
    km = platform.units.get("km")
except Unit.NotFound:
    km = platform.units.create("km")

# Distance in thousands of miles
d = run.optimization.parameters.create("d", constrained_to_indexsets=["i", "j"])
# You can start with some data ...
d_data = {
    "i": ["seattle", "seattle", "seattle", "san-diego"],
    "j": ["new-york", "chicago", "topeka", "new-york"],
    "values": [2.5, 1.7, 1.8, 2.5],
    "units": [km.name] * 4,
}
d.add(d_data)

# ... and expand it later on:
d.add({"i": ["san-diego"], "j": ["chicago"], "values": [1.8], "units": ["km"]})
d.add({"i": ["san-diego"], "j": ["topeka"], "values": [1.4], "units": ["km"]})

Every time you add data, though, **all** columns must be present!

In [ ]:
# Cost per case per 1000 miles

# TODO we could really use a units.get_or_create() function!
try:
    unit_cost_per_case = platform.units.get("USD/km")
except Unit.NotFound:
    unit_cost_per_case = platform.units.create("USD/km")

f = run.optimization.scalars.create(name="f", value=90, unit=unit_cost_per_case)

### Defining `Variable`s and `Equation`s in the scenario

The levels and marginals of these variables and equations will be imported to the scenario when reading the model solution.

In [ ]:
# Initialize the decision variables and equations
z = run.optimization.variables.create("z")
x = run.optimization.variables.create("x", constrained_to_indexsets=["i", "j"])
supply = run.optimization.equations.create("supply", constrained_to_indexsets=["i"])
demand = run.optimization.equations.create("demand", constrained_to_indexsets=["j"])

### Solve the scenario

In this tutorial, we solve the scenario using GAMS. 
GAMS requires model data and structure to follow their proprietary formats, so we need to write out model data to this format before GAMS can handle it ("`data_file`").
For this tutorial, the structure of the model (the equations that relate the variables) has already been written to a `transport_ixmp4.gms`, so we can simply use that ("`model_file`").
After the calculation, GAMS writes the results out in the `.gdx` format again, which we need to read back into our `Run` ("`result_file`").
Thus, the solution process requires three steps:

1. Write the `Run` data to a `.gdx` file that GAMS can read.
2. Solve the scenario in GAMS.
3. Read the solution data back into our `Run`.

For convenience, these are pre-existing functions here, so we can immediately use them. Feel free to check out their implementation in `dantzig_model_gams.py`, though, if you are interested!

In [ ]:
from pathlib import Path

from tutorial.transport.dantzig_model_gams import (
    read_solution_to_run,
    solve,
    write_run_to_gams,
)

# First, we collect all relevant Run data in a gams container:
gams_container = write_run_to_gams(run=run)

# For the next steps, you can adapt the locations of the files as discussed above.
# If you are not sure how, please use these defaults:
data_file = Path("transport_data.gdx").absolute()
model_file = Path("transport_ixmp4.gms").absolute()
result_file = Path("transport_results.gdx").absolute()

# The gams container then allows us to write the required data format file:
gams_container.write(write_to=data_file)

# Now, we can actually solve the model:
solve(model_file=model_file, data_file=data_file, result_file=result_file)

# Finally, we read the solution back into our Run:
read_solution_to_run(run=run, result_file=result_file)

### Display and analyze the results

In [ ]:
# Display the objective value of the solution
z.levels

In [ ]:
# Display the quantities transported from canning plants to demand locations
x.data

In [ ]:
# Display the quantities and marginals (shadow prices) of the demand balance constraints
demand.data

In [ ]:
# Display the quantities and marginals (shadow prices) of the supply balance constraints
supply.data